In [2]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix
import pickle
from tqdm import tqdm
import multiprocessing as mp
import random
#gateset1 [1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
#gateset2 [16192, 11846, 15055, 13880, 10476, 10481]
###rb_seq_deompose
with open('rb_seq_decompose_gateset1.pkl', 'rb') as f:
    rb_seq_decompose = pickle.load(f)
###primitive gate PTM
with open('P_PTM_gateset1.pkl', 'rb') as f:
    P_PTM = pickle.load(f)
#### 2-qubit pauli basis
pauli_basis=[np.asarray(np.kron(i,j)) for i in [I, X, Y, Z] for j in [I, X, Y, Z]]
#### 2-qubit |0> and |1>
e_state=np.kron(np.asarray([[0,0],[0,1]]),np.asarray([[0,0],[0,1]]))
g_state=np.kron(np.asarray([[1,0],[0,0]]),np.asarray([[1,0],[0,0]]))
####Density matrix to PTM
def densityop_01_2_pauli(density_op):
    #assert density_op.ndim !=4,'wrong dim'
    A=np.asarray([np.trace(density_op@i_den) for i_den in pauli_basis]).reshape(density_op.size,1)
    return A
def densityop_pauli_2_01(density_op):
    #assert density_op.shape == (len(density_op),1),'wrong dim'
    A_den=np.asarray([density_op[i_den]*pauli_basis[i_den] for i_den in range(16)])
    B_den=np.zeros((4,4))
    for i_den in A_den:
        B_den=B_den+i_den
    return B_den/4
def dephaseing_PTM(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0]:
        A[i_deph] = A[i_deph]*np.sqrt(p+(1-p)/4)
    for i_deph in [1,2,3]:
        A[i_deph] = A[i_deph]*np.sqrt((1-p)/4)
#########kraus to PTM 
    return kraus2pauli_liouville(A)
def depolarizing_PTM(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-15*p/16)
    for i_depo in range(1,len(A)):
        A[i_depo]=A[i_depo]*np.sqrt(p/16)
#########kraus to PTM        
    return kraus2pauli_liouville(A)
def sequence_to_probability(_rb_seq):
    random.seed()
    # truncate Gaussian distribute p
    _lower_bound = 0
    _upper_bound = 1
    my_mean = noise_strength
    my_std = my_mean*deviation
    _a, _b = (_lower_bound - my_mean) / my_std, (_upper_bound - my_mean) / my_std
    _random_devi = truncnorm.rvs(_a, _b, len(_rb_seq))
####initial state    
    _state=densityop_01_2_pauli(g_state)
####depo or deph=  noise channel @ Cliff @ state
    if noise_channel==0:
        for _i in _rb_seq :
            _state=depolarizing_PTM[_random_devi]@P_PTM[_i]@_state
    elif noise_channel==1:
        for _i in _rb_seq :
            _state=dephaseing_PTM[_random_devi]@P_PTM[_i]@_state
#### diagonal density matrix
    _state = np.diagonal(densityop_pauli_2_01(_state))
####get prob = [00]*(1-err)+ [01]*err/3 + [10]*err/3 + [11]*err/3
    _state_prob = _state[0]*(1-spam_error)+sum(_state[_i]*spam_error/3 for _i in [1,2,3] )

    print(_state_prob)
    return random.choices([True, False], weights=[_state_prob,1-_state_prob], k=1)[0]
if __name__ == '__main__':

####### input
    rb_types=int(input('-1 rb, 0 decomp_rb, 1 prb\n'))
    max_seq= int(input('max seq \n'))
    spam_error= float(input('SPAM error \n'))
    noise_channel= int(input('depolarizing (0) or dephasing (1) \n'))
    noise_strength= float(input('noisy(0)-----(1)noiseless\n'))
    deviation = float(input('deviation p (%)\n'))*0.01
    repeat = int(input('repeat number\n'))
####### rb gate number i    
    for i in tqdm([i for i in range(1,max_seq)],ascii=True,desc="sequence_length",dynamic_ncols=True):
####### rb sequence into probablility
        types_probability=[]
        m_average=[]
        pool = mp.Pool()
        repeat_probability = pool.map(sequence_to_probability,[ e*repeat for e in rb_seq_decompose[i]])
        pool.close()
        pool.join()

sequence_length:   0%|          | 0/1 [00:09<?, ?it/s]


ZeroDivisionError: float division by zero

In [8]:
#prb seq w/inverse
import random
from functools import reduce
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import Pool
#### 2-qubit pauli basis
with open('H_P_CNOT_kruas.pkl', 'rb') as f:
    Primitive_kruas = pickle.load(f)
#### weighted
Primitive_weights=[15694, 12780, 20232, 15752, 12124, 10492]

def prb_seq_fun(seq_number):
    random.seed()
    prb_seq_index= random.choices([_i for _i in range(len(Primitive_kruas))], weights=Primitive_weights, k=seq_number)
    prb_seq_kraus=reduce(lambda x, y : y@x, [Primitive_kruas[_i] for _i in prb_seq_index])

    for i_rb in range(len(Primitive_kruas)):
        if (np.isclose(abs(np.trace(Primitive_kruas[i_rb]@prb_seq_kraus)),4)):
            prb_seq_index.append(i_rb)
            return prb_seq_index
        elif i_rb==len(Primitive_kruas):
            return None
if __name__ == '__main__':
    prb_seq={}
    max_seq_number=int(input('max sequence number'))
    types=int(input('different types sequence '))
    for i in trange(1,max_seq_number):
########init_        
        prb_seq[i]=[0]
#        while(len(prb_seq[i])<types and j<100):
        pool=Pool()
        _prb_seq=pool.map(prb_seq_fun,[i]*types*100)
        prb_seq[i]=list(filter(None,_prb_seq))
        pool.close()
        pool.join()

'''    with open('prb_seq.pkl', 'wb') as f:
        pickle.dump(prb_seq, f)'''

100%|██████████| 9/9 [00:53<00:00,  5.91s/it]


"    with open('prb_seq.pkl', 'wb') as f:\n        pickle.dump(prb_seq, f)"

In [9]:
print(prb_seq)

{1: [[1, 1], [0, 0]], 2: [], 3: [[4, 0, 0, 4], [4, 1, 1, 4]], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}


In [10]:
#transform rb sequence into prb sequence 
import pickle
import itertools
with open('rb_seq.pkl', 'rb') as f:
    rb_seq = pickle.load(f)

with open('Cliff_index_gateset1.pkl', 'rb') as f:
    decomp = pickle.load(f)

c={}
f={}
for i in rb_seq:
    b=[]
    for j in rb_seq[i]:
        a=[]
        for k in j:
            a.append(decomp[k])
        e=list(itertools.chain.from_iterable(a))
        b.append(e)
    c[i]=b
with open('rb_seq_gateset1.pkl', 'wb') as f:
    pickle.dump(c, f)


In [2]:
#rb seq w/ inverse
from functools import reduce
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import  Pool
### 2-qubit Clifford matrix
with open('Cliff_kruas.pkl', 'rb') as f:
    Cliff_kruas = pickle.load(f)
###generate rb sequence
def rb_seq_fun(seq_number):
    np.random.seed()
### random choose 11520 Clifford matrix
    rb_seq_index= np.random.randint(low=0, high=11520, size=seq_number).tolist()
### rb sequence matrix
    rb_seq_kraus=reduce(lambda x, y : y@x, [Cliff_kruas[_i] for _i in rb_seq_index])
### find rb inverse element
    for i_rb in range(11520):
        if (np.isclose(abs(np.trace(Cliff_kruas[i_rb]@rb_seq_kraus)),4)):
            rb_seq_index.append(i_rb)
            return rb_seq_index
        elif i_rb == 11519:
            return False
if __name__ == '__main__':
    rb_seq={}
    max_seq_number=int(input('max sequence number'))
    types=int(input('different types sequence '))
    for i in trange(1,max_seq_number):
        pool=Pool()
        rb_seq[i]=pool.map(rb_seq_fun,[i]*types)
        assert False not in rb_seq[i], 'false in rbseq'
        pool.close()
        pool.join()
    with open('rb_seq.pkl', 'wb') as f:
        pickle.dump(rb_seq, f)
        

100%|██████████| 1/1 [00:00<00:00,  1.33it/s]


In [7]:
a=[1, 3, 8, 13, 14, 7, 1, 2, 7, 14, 7, 13, 1, 8, 4, 13, 7, 1, 8, 2, 14, 10, 1, 2, 13, 1, 14, 10, 1, 7, 1, 14, 7, 14, 4, 1, 7, 8, 2, 9, 13, 14, 1, 7, 10, 13, 14, 1, 9, 13, 1, 14, 1, 4, 13, 8, 14, 2, 7, 14, 2, 3, 8, 2, 4, 9]
#check rb seq
from functools import reduce
import numpy as np
import pickle
from print_large import _print_big_matrix
from tqdm import trange
from multiprocessing import  Pool
with open('Cliff_kruas.pkl', 'rb') as f:
    c = pickle.load(f)
with open('P_kruas_gateset1.pkl', 'rb') as f:
    p = pickle.load(f)

print(abs(np.trace(reduce(lambda x,y: y@x,[p[i] for i in a]))))

3.999999999999995
